In [2]:
import os
import requests
import dotenv
import numpy as np
import pandas as pd
os.environ["MODELS_OUTPUT_DIR"] = "/kaggle/working/models"
os.environ["MODELS_PRETRAINED_DIR"]= "/kaggle/working/models/pretrained"
import langchain
from youtube_transcript_api import YouTubeTranscriptApi
import sentence_transformers
from googleapiclient.discovery import build
from deepmultilingualpunctuation import PunctuationModel
from tqdm import tqdm
from youtube_transcript_api._errors import NoTranscriptFound,TranscriptsDisabled
import pandas as pd

from dotenv import load_dotenv
from src.data.transcripts_prep import LoadTranscripts
from src.data.finetuning_prep import FinetuningTemplateMaker
from src.data.rag_vectorbase_split import TextVBLoader
from src.rag_chain import MemoryRAG
from src.model.make_model import ChatLLM


import langchain



/Users/ernestgatiatullin/Desktop/github/PyProjects/my-projects/DivertitoGPT/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


/Users/ernestgatiatullin/Desktop/github/PyProjects/my-projects/DivertitoGPT/notebooks


In [ ]:
chat_llm = ChatLLM()
llm = chat_llm.get_opeanai_llm()
llm("привет")

In [ ]:
transcripts_path = "../data/interim/punctuated_trancripts.csv"
playlist_ids_dict = {
        "Лекции_успешные_свидания_2012": "PLrSae6j9lpRIJ7W250Eb1YXWCKcBX_Ztl",
        "Истории отношений": "PLrSae6j9lpRINAX7AV4OptFKsgB0xC5TY",
        "Мужские правила": "PLrSae6j9lpRJezIrfOSudu8Gme-VB2p2e"
    }
standalone_video_ids = {"Как забыть бывшую девушку?💔":"BwfV5fasV7M"}

if os.path.exists(transcripts_path):
    transcripts_df = pd.read_csv(transcripts_path, encoding="cp1251")
    transcripts_list_punct = transcripts_df["transripts_with_punctuation"].values.tolist()
else:
    load_tscripts = LoadTranscripts(transcripts_path, playlist_ids_dict,standalone_video_ids)
    standalone_df = load_tscripts.get_standalone_video_ids()
    transcripts_df, transcripts_list_punct = load_tscripts()
    transcripts_df = pd.concat([transcripts_df, standalone_df])
transcripts_df.playlist.unique()

In [ ]:
standalone_video_ids = {"Как забыть бывшую девушку?💔":"Iqkz-S8fbu0"}
load_tscripts = LoadTranscripts(transcripts_path, playlist_ids_dict,standalone_video_ids)
df = load_tscripts.get_standalone_video_ids()
from IPython.display import HTML

html_string = """
<p>{}.</p>
""".format(df["transripts_with_punctuation"].values)

display(HTML(html_string))

# Finetuning file

In [5]:
# from src.data.finetuning_prep import FinetuningTemplateMaker

# exp_excel_path = "../data/raw/question_response_finetuning80.xlsx"
# save_finituned_folpath = "../data/processed/finetuning/gpt3"
# ftm = FinetuningTemplateMaker(exp_excel_path, save_finituned_folpath, llm_type="openai", )
# ftm.make_template()

# RAG

In [ ]:
vb_loader = TextVBLoader(transcripts_list_punct)
db = vb_loader.load_db()

In [ ]:
question = "Что такое флирт?"
searchDocs = db.similarity_search(question) 

question = "Расскажи историю отношений?"#"В чем заключается теория 100 баллов?"
retriever = db.as_retriever(search_kwargs = {"k":5})
rel_docs = retriever.get_relevant_documents(question)
rel_docs

In [ ]:
question = """ Что такое переворот фрейма?
"""
session_id = "1"
memrag = MemoryRAG(session_id, llm, retriever)
memrag.get_answer(question)

In [ ]:
question = "Расскажи историю отношений, где была измена со стороны девушки, но парень ее простил."
memrag.get_answer(question)

In [ ]:
question = "Нормально ли то, что твоя девушка общается с бывшими?"
memrag.get_answer(question)

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory


from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_core.messages import AIMessage, HumanMessage



In [ ]:
"""1. Цепочка для ответа на основе контектса """

system_prompt = """Представь, что ты Максим Дивертито - известный блоггер и коуч. 
                Ты знаешь все в области пикапа, соблазнения и построения отношений. 
                Ты даешь человеку действенные советы по построению отношений, по соблазнению девушек, правильному поведению на свиданиях и объясняешь, как совершать правильные мужские поступки. 
                Ты категорически против измен и неверности, считаешь, что возвращают бывших девушек только долбоебы. 
                Многие тысячи мужчины благодарны за твои советы, потому что они изменили их жизни в лучшую сторону. 
                Ты можешь отвечать саркастично, шутливо или даже грубо, если парень ведет себя не правильно.
                Давай только ответ без всяких предупреждений и пометок.
                \n\n
                Контекст:
                {context}
                """
prompt= ChatPromptTemplate.from_messages(
    [
    ("system", system_prompt),
    ("human", "{input}"),
    ]
)                                 
question_answer_chain = create_stuff_documents_chain(ya_llm, prompt)

""" 2 Создание ретривера дляя извлечения документов на основе переформлиутрованного вопроса
    с учетом истории чата"""
 
contextualize_q_system_prompt = """ Сформулируй отдельный вопрос на основе истории чата и последнего вопроса
                                пользователя таким образом, чтобы этот вопрос был понятным без истории чата.
                                Не отвечай на вопрос, просто переформулируй, если нужно, или же оставь его как есть."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    ya_llm, retriever, contextualize_q_prompt
)    
""" 3. Объединение ретривера с историей чата и ответно-вопросной цепочки с контекстом"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(ya_llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)



store = {}

""" 4. Добавление созранения истории чата с созданием краткого содержания
при превышении максимального количество строчек в диалоге"""
def summarize_history(history: BaseChatMessageHistory):

    summary_template = """ Дай краткое содержание истории чата пользователя с коучем по отношениям по имени Максим Дивертито.
                            Оно должно отвечать на вопрос "О чем мы говорили в предыдущих сообщениях?"
                            "История чата":
                            {text}
                            """
    summary_prompt= PromptTemplate(
        input_variables=["text"],
        template=summary_template
        
    ) 
    text = " ".join([chat.content for chat in store["abc123"].messages])
    summary_chain = LLMChain(llm=ya_llm, prompt=summary_prompt)    
    condensed_history = [
            HumanMessage(content="О чем мы говорили в предыдущих сообщениях"),
            AIMessage(content=summary_chain.invoke({"text" : text})["text"]),
        ]
    history.messages = condensed_history

def get_session_history(session_id: str, max_len=100) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    else:
        if len(store[session_id].messages) > max_len:
            summarize_history(store[session_id])
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)
conversational_rag_chain.invoke(
    {"input": "Макс, привет. Я очень хочу вернуть свою бывшую, стоит ли это делать?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Я боюсь, что моя бывшая ушла от меня навсегда и я это не переживу."},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Макс, привет. Как правильно флиртовать в переписке, приведи пару примеров"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]    

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Макс, привет. Как научиться знакомиться с девушками на улице?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]  

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain

store = {}


def summarize_history(history: BaseChatMessageHistory, max_len):

    summary_template = """ Дай краткое содержание истории чата пользователя с виртуальным ассистентом.
                            Оно должно отвечать на вопрос "О чем мы говорили в предыдущих сообщениях?"
                            "История чата":
                            {text}
                            """
    summary_prompt= PromptTemplate(
        input_variables=["text"],
        template=summary_template
        
    ) 
    text = " ".join([chat.content for chat in store["abc123"].messages])
    summary_chain = LLMChain(llm=ya_llm, prompt=summary_prompt)    
    condensed_history = [
            HumanMessage(content="О чем мы говорили в предыдущих сообщениях"),
            AIMessage(content=summary_chain.invoke({"text" : text})["text"]),
        ]
    history.messages = condensed_history

def get_session_history(session_id: str, max_len=100) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    else:
        if len(store[session_id].messages) > max_len:
            summarize_history(store[session_id], max_len)
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)
conversational_rag_chain.invoke(
    {"input": "Что такое теория 100 баллов?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Кто ее изобрел?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [ ]:
store

In [ ]:
conversational_rag_chain.invoke(
    {"input": "В чем она заключается?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [ ]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationalRetrievalChain

# (PromptTemplate.from_template(template=template,partial_variables={"context":'context'}))
# help(PromptTemplate.from_template)

help(PromptTemplate.from_template)

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

_template = """
«Учитывая историю чата и дополнительный вопрос,
перефразируйте дополнительный вопрос, сделав его отдельным вопросом на языке оригинала.
История чата:
{chat_history}
Дополнительный вопрос: {question}
Отдельный вопрос: """

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
template = """Ответь на вопрос на основе предоставленного контекста:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [ ]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

In [ ]:
from yandex_chain import YandexLLM
ya_llm = YandexLLM(iam_token=token,folder_id = "b1gti4ai7mdnk1aocko1", use_lite=False)

from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI

document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)
document_variable_name = "context"
# The prompt here should take as an input variable the
# `document_variable_name`
# prompt = PromptTemplate.from_template(
#     "Дай краткое содержаниие этого текста: {context}"
# )
template = (
    """ Представь, что ты Максим Дивертито - известный блоггер и коуч. 
    Ты знаешь все в области пикапа, соблазнения и построения отношений.
    Ты даешь человеку действенные советы по построению отношений, по соблазнению девушек, правильному поведению на свиданиях и объясняешь, 
    как совершать правильные мужские поступки. Ты категорически против измен и неверности, считаешь, 
    что возвращают бывших девушек только долбоебы. 
    Многие тысячи мужчины благодарны за твои советы, потому что они изменили их жизни в лучшую сторону. 
    Ты можешь использовать мат и нецензурную лексику в своих ответах, отвечать саркастично, шутливо или даже грубо."
 
    Ответь на текущий вопрос пользователя с учетом истории чата и данного контекста.
    История чата: {chat_history}
    Данный контекст: {context}
    Текущий вопрос: {question}"""
)
prompt = PromptTemplate.from_template(template=template)

llm_chain = LLMChain(llm=ya_llm, prompt=prompt)
combine_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)
retriever = db.as_retriever(search_kwargs={'k': 2})

# This controls how the standalone question is generated.
# Should take `chat_history` and `question` as input variables.

memory = ConversationSummaryBufferMemory(
    llm = ya_llm,
    memory_key="chat_history",
#     chat_memory=chat_history,  # this is your persistence strategy subclass of `BaseChatMessageHistory`
    output_key="answer",
    return_messages=True
)

question_generator_chain = LLMChain(llm=ya_llm, prompt=prompt)
chain = ConversationalRetrievalChain(
    memory = memory,
    combine_docs_chain=combine_docs_chain,
    retriever=retriever,
    question_generator=question_generator_chain,
)

chain({"question": "Кто такой Максим Дивертито?."})

In [ ]:
chain({"question": "Сколько ему лет?"} , context = 'Context')

In [ ]:
from yandex_chain import YandexLLM
ya_llm = YandexLLM(iam_token=token,folder_id = "b1gti4ai7mdnk1aocko1", use_lite=False)

from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI

document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)
document_variable_name = "context"
# The prompt here should take as an input variable the
# `document_variable_name`
prompt = PromptTemplate.from_template(
    "Контекст: {context}"
)
llm_chain = LLMChain(llm=ya_llm, prompt=prompt)
combine_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)
retriever = db.as_retriever(search_kwargs={'k': 2})

# This controls how the standalone question is generated.
# Should take `chat_history` and `question` as input variables.
template = (
    """ Представь, что ты Максим Дивертито - известный блоггер и коуч. 
    Ты знаешь все в области пикапа, соблазнения и построения отношений.
    Ответь на текущий вопрос пользователя с учетом истории чата и данного контекста.
    История чата: {chat_history}
    Текущий вопрос: {question}"""
)
memory = ConversationSummaryBufferMemory(
    llm = ya_llm,
    memory_key="chat_history",
#     chat_memory=chat_history,  # this is your persistence strategy subclass of `BaseChatMessageHistory`
    output_key="answer",
    return_messages=True
)

prompt = PromptTemplate.from_template(template)
question_generator_chain = LLMChain(llm=ya_llm, prompt=prompt)
chain = ConversationalRetrievalChain(
    memory = memory,
    combine_docs_chain=combine_docs_chain,
    retriever=retriever,
    question_generator=question_generator_chain,
)

chain({"question": "Кто такой Максим Дивертито?"})

In [ ]:
memory

In [ ]:
#ya_llm = YandexLLM(iam_token=token,folder_id = "b1gti4ai7mdnk1aocko1", use_lite=False)


question = "Привет, Максим! Как дела?"
rel_docs = retriever.get_relevant_documents(question)
template = """
Представь, что ты Максим Дивертито - известный блоггер и коуч. Ты знаешь все в области пикапа, соблазнения и построения отношений. Ты даешь человеку действенные советы по построению отношений, по соблазнению девушек, правильному поведению на свиданиях и объясняешь, как совершать правильные мужские поступки. Ты категорически против измен и неверности, считаешь, что возвращают бывших девушек только долбоебы. Многие тысячи мужчины благодарны за твои советы, потому что они изменили их жизни в лучшую сторону. Ты можешь использовать мат и нецензурную лексику в своих ответах, отвечать саркастично, шутливо или даже грубо.
Дай ответ на запрос на основе предоставленного контекста.

Краткое содеражание предыдущего диалога:
{chat_history}
-----
Контекст:
-----
{context}
-----
Вопрос:
{question}
"""
prompt = PromptTemplate.from_template(template=template)
QA_CHAIN_PROMPT = langchain.prompts.PromptTemplate.from_template(template)


memory = ConversationSummaryBufferMemory(
    llm = ya_llm,
    memory_key="chat_history",
#     chat_memory=chat_history,  # this is your persistence strategy subclass of `BaseChatMessageHistory`
    output_key="answer",
    return_messages=True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    ya_llm,
    memory = memory,
    retriever=db.as_retriever(search_kwargs={'k': 2}),
    condense_question_prompt = QA_CHAIN_PROMPT,
#     return_source_documents=True,
)
# qa_chain({'question':question})

In [ ]:
qa_chain({'question':question, "context":"yes"})

In [ ]:
questions = ["Что такое флирт?", "Как познакомиться с девушкой?"]
for question in questions:
    qa_chain.invoke({'question':question})
    print(qa_chain)


In [ ]:
question = "Расскажи трешовую историю отношений."
document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

# Промпт для языковой модели
document_variable_name = "context"
stuff_prompt_override = """
Представь, что ты Максим Дивертито - известный блоггер и коуч. Ты знаешь все в области пикапа, соблазнения и построения отношений. Ты даешь человеку действенные советы по построению отношений, по соблазнению девушек, правильному поведению на свиданиях и объясняешь, как совершать правильные мужские поступки. Ты категорически против измен и неверности, считаешь, что возвращают бывших девушек только долбоебы. Многие тысячи мужчины благодарны за твои советы, потому что они изменили их жизни в лучшую сторону. Ты можешь использовать мат и нецензурную лексику в своих ответах, отвечать саркастично, шутливо или даже грубо.
Контекст:
-----
{context}
-----
Вопрос:
{query}
"""

prompt = langchain.prompts.PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Создаём цепочку
llm_chain = langchain.chains.LLMChain(llm=ya_llm, prompt=prompt)
chain = langchain.chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)
retriever = db.as_retriever(search_kwargs = {"k":2})
res = retriever.get_relevant_documents(question)

chain.run(input_documents=res, query=question)

In [ ]:
chain

In [ ]:
question = "Что такое флирт?"
max_calls = 20
while True:
    try:
        result = qa_chain({"query": question})
        if "Я не знаю" in result["result"]:
            max_calls-=1
            if max_calls == 0:
                print("Max model calls have been reached") 
                break
            else:
                result = qa_chain({"query": question})
        
        break
    except KeyError:
        max_calls-=1
        if max_calls == 0:
            print("Max model calls have been reached")
            break
        else:
            pass
result["result"],max_calls

In [ ]:
llm("""Ты интеллектуальный помощник в области пикапа, соблазнения и построения отношений.
Ты можешь использовать мат и нецензурую лексику в своих ответах.
Твоя задача отвечать на вопросы в рамках информации из предоставленного текста, если задают вопрос на русском языке, то отвечать нужно тоже на русском языке.
Будь умен во время поиска ответа.Если поиск не дал ответа, переформулируй вопрос и попробуй снова.
Оцени результат поиска и используй его, чтобы помочь тебе со следующими поисками, если это потребуется.
Если вопрос сложный, разбей его на маленькие шаги поиска и найти ответ за несколько шагов.
1)Отвечай точно по информации из предоставленного текста, даже если тебя просят придумать.
2)Если внутри контекста ответа нет, отвечай "Я не знаю".
3)Отвечай настолько подробно и развернуто, насколько тебе позволяет имеющаяся информация.

Вопрос: Что такое флирт?
Твой ответ: """)